In [3]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer

# Step 1: JSON에서 메타데이터 로드
with open('metadata.json', 'r') as f:
    metadata = json.load(f)

# Step 2: Value 데이터 추출 및 임베딩 생성
model = SentenceTransformer('all-MiniLM-L6-v2')  # 임베딩 모델
values = [item['value'] for item in metadata.values()]  # Value만 추출
vectors = model.encode(values)  # Value를 벡터로 변환

# Step 3: FAISS 인덱스 생성
index = faiss.IndexFlatL2(vectors.shape[1])  # L2 거리 기반 인덱스
index.add(np.array(vectors))  # 벡터 추가

# Step 4: FAISS 인덱스 파일 저장
faiss.write_index(index, 'faiss_index.index')
print("FAISS 인덱스 파일이 성공적으로 생성되었습니다.")

FAISS 인덱스 파일이 성공적으로 생성되었습니다.


In [4]:
import faiss
import json
from sentence_transformers import SentenceTransformer

# Step 1: FAISS 인덱스 및 메타데이터 로드
index = faiss.read_index('faiss_index.index')  # FAISS 인덱스 로드
with open('metadata.json', 'r') as f:
    metadata = json.load(f)

# Step 2: 검색 쿼리 정의 및 임베딩 생성
query = "유형 A에 속하는 질문"  # 검색할 문장
model = SentenceTransformer('all-MiniLM-L6-v2')  # 임베딩 모델
query_vector = model.encode([query])  # 쿼리 임베딩 생성

# Step 3: 유사도 검색 수행
distances, indices = index.search(query_vector, k=1)  # 가장 가까운 1개 검색

# Step 4: 검색 결과에 따라 메타데이터 조회
for idx in indices[0]:
    result = metadata[str(idx)]  # JSON에서 인덱스 기반으로 메타데이터 조회
    print(f"검색된 Key: {result['key']}")
    print(f"검색된 Value: {result['value']}")

검색된 Key: 문제 유형 A
검색된 Value: 이 문장은 유형 A에 해당하는 문제입니다.
